# An sample to finetune Baichuan2-7B-Base distruibutely on SageMaker

**To avoid 'no left space' error, move the docker root directory**

sudo systemctl stop docker

sudo systemctl stop docker.socket 

sudo mv /var/lib/docker /home/ec2-user/SageMaker 

sudo ln -s /home/ec2-user/SageMaker/docker /var/lib/docker 

sudo systemctl start docker.socket

sudo systemctl start docker

In [ ]:
## Update sagemaker python sdk version
!pip install -U sagemaker

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role


sess                     = sagemaker.Session()
role                     = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()

account                  = sess.boto_session.client("sts").get_caller_identity()["Account"]
region                   = sess.boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:
%%script bash

rm -rf s5cmd*
wget https://github.com/peak/s5cmd/releases/download/v2.1.0/s5cmd_2.1.0_Linux-64bit.tar.gz
tar xvzf s5cmd_2.1.0_Linux-64bit.tar.gz

rm -rf src
mkdir src
cp s5cmd src/
cd src

# The version is 2023.09.09
git clone https://github.com/baichuan-inc/Baichuan2.git
cd Baichuan2
git reset --hard f620769c667ce6380770398af1d8449e775ec271

## Download pretrained model from HuggingFace Hub

To avoid download model from Huggingface hub failure, we download first and push those model files to S3 bucket first.

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path


local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "baichuan-inc/Baichuan2-7B-Base"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model", "*.py", "*.txt"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    allow_patterns=allow_patterns,
    revision='f2cc3a689c5eba7dc7fd3757d0175d312d167604'
)

**Upload model files to S3**

In [ ]:
# Get the model files path
import os
from glob import glob

local_model_path = None

paths = os.walk(r'./model')
for root, dirs, files in paths:
    for file in files:
        if file == 'config.json':
            print(os.path.join(root,file))
            local_model_path = str(os.path.join(root,file))[0:-11]
            print(local_model_path)
if local_model_path == None:
    print("Model download may failed, please check prior step!")

In [ ]:
%%script env sagemaker_default_bucket=$sagemaker_default_bucket local_model_path=$local_model_path bash

chmod +x ./s5cmd
./s5cmd sync ${local_model_path} s3://${sagemaker_default_bucket}/llm/models/baichuan2/baichuan-inc/Baichuan2-7B-Base/ 

rm -rf model

## Prepare docker image

In [1]:
!git clone https://github.com/aws-samples/sagemaker-stablediffusion-quick-kit.git

Cloning into 'sagemaker-stablediffusion-quick-kit'...
remote: Enumerating objects: 897, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 897 (delta 112), reused 122 (delta 99), pack-reused 698
Receiving objects: 100% (897/897), 10.90 MiB | 34.99 MiB/s, done.
Resolving deltas: 100% (450/450), done.


In [2]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
# From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

RUN pip3 install deepspeed==0.10.0 \
    && pip3 install transformers==4.29.2 \
    && pip3 install jsonlines==3.1.0 \
    && pip3 install accelerate==0.22.0 \
    && pip3 install xformers==0.0.21 \
    && pip3 install peft==0.5.0 \
    && pip3 install wandb

COPY src/Baichuan2/fine-tune/requirements.txt .
RUN pip3 install -r requirements.txt

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

Overwriting Dockerfile


In [3]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


**Build image and push to ECR.**

In [4]:
## define repo name, should contain *sagemaker* in the name
repo_name="sagemaker-baichuan2-7b-base-finetune"

In [5]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  45.54MB
Step 1/8 : From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04
 ---> 1f37d018af76
Step 2/8 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> 9cdd0c4b4c35
Step 3/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 7d0700ee0c80
Step 4/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> f0079091e634
Step 5/8 : RUN pip3 install deepspeed==0.10.0     && pip3 install transformers==4.29.2     && pip3 install jsonlines==3.1.0     && pip3 install accelerate==0.22.0     && pip3 install xformers==0.0.21     && pip3 install peft==0.5.0     && pip3 install wandb
 ---> Running in deff3140f331
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.6/836.6 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for deepspeed: filename=deepspeed-0.10.0-py3-none-any.whl size=8774

**Generate training entrypoint script.**

**Note: DO NOT CHANGE BELOW VAlUE OF "output_dir" and "cache_dir", keep it "/tmp/llama_out" and "/tmp".**

Below is just a testing to fine-tune on a sample dataset (just 8 samples), you could change ```data_path``` to your dataset for furthur fine tune.

For the dataset download, you could follow the way how to download pretrain model:
```
./s5cmd sync s3://$MODEL_S3_BUCKET/llm/models/llama/pinkmanlove/llama-7b-hf/* /tmp/llama_pretrain/
```

It is recommend to use the folder ```/tmp/dataset/```.

## Notice

We modified some parts of ```Baichuan2/fine-tune/fine-tune.py```, such as how to save model.

The version is 2023-09-09

In [2]:
!mv src/Baichuan2/fine-tune/fine-tune.py src/Baichuan2/fine-tune/fine-tune.py.bk

In [3]:
%%writefile src/Baichuan2/fine-tune/fine-tune.py

import os
import math
import pathlib
from typing import Optional, Dict
from dataclasses import dataclass, field
import json

import torch
from torch.utils.data import Dataset
import transformers
from transformers.training_args import TrainingArguments


@dataclass
class ModelArguments:
    model_name_or_path: Optional[str] = field(default="baichuan-inc/Baichuan2-7B-Base")


@dataclass
class DataArguments:
    data_path: str = field(
        default=None, metadata={"help": "Path to the training data."}
    )


@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(
        default=512,
        metadata={
            "help": "Maximum sequence length. Sequences will be right padded (and possibly truncated)."
        },
    )
    use_lora: bool = field(default=False)


class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(
        self,
        data_path,
        tokenizer,
        model_max_length,
        user_tokens=[195],
        assistant_tokens=[196],
    ):
        super(SupervisedDataset, self).__init__()
        self.data = json.load(open(data_path))
        self.tokenizer = tokenizer
        self.model_max_length = model_max_length
        self.user_tokens = user_tokens
        self.assistant_tokens = assistant_tokens
        self.ignore_index = -100
        item = self.preprocessing(self.data[0])
        print("input:", self.tokenizer.decode(item["input_ids"]))
        labels = []
        for id_ in item["labels"]:
            if id_ == -100:
                continue

            labels.append(id_)
        print("label:", self.tokenizer.decode(labels))

    def __len__(self):
        return len(self.data)

    def preprocessing(self, example):
        input_ids = []
        labels = []

        for message in example["conversations"]:
            from_ = message["from"]
            value = message["value"]
            value_ids = self.tokenizer.encode(value)

            if from_ == "human":
                input_ids += self.user_tokens + value_ids
                labels += [self.tokenizer.eos_token_id] + [self.ignore_index] * len(
                    value_ids
                )
            else:
                input_ids += self.assistant_tokens + value_ids
                labels += [self.ignore_index] + value_ids
        input_ids.append(self.tokenizer.eos_token_id)
        labels.append(self.tokenizer.eos_token_id)
        input_ids = input_ids[: self.model_max_length]
        labels = labels[: self.model_max_length]
        input_ids += [self.tokenizer.pad_token_id] * (
            self.model_max_length - len(input_ids)
        )
        labels += [self.ignore_index] * (self.model_max_length - len(labels))
        input_ids = torch.LongTensor(input_ids)
        labels = torch.LongTensor(labels)
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id)
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
        }

    def __getitem__(self, idx) -> Dict[str, torch.Tensor]:
        return self.preprocessing(self.data[idx])


def train():
    parser = transformers.HfArgumentParser(
        (ModelArguments, DataArguments, TrainingArguments)
    )
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        trust_remote_code=True,
        cache_dir=training_args.cache_dir,
    )
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        use_fast=False,
        trust_remote_code=True,
        model_max_length=training_args.model_max_length,
        cache_dir=training_args.cache_dir,
    )
    if training_args.use_lora:
        from peft import LoraConfig, TaskType, get_peft_model

        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            target_modules=["W_pack"],
            inference_mode=False,
            r=1,
            lora_alpha=32,
            lora_dropout=0.1,
        )
        model.enable_input_require_grads()
        model = get_peft_model(model, peft_config)
        model.print_trainable_parameters()

    dataset = SupervisedDataset(
        data_args.data_path, tokenizer, training_args.model_max_length
    )
    trainer = transformers.Trainer(
        model=model, args=training_args, train_dataset=dataset, tokenizer=tokenizer
    )
    trainer.train()
    # trainer.save_state()
    # trainer.save_model(output_dir=training_args.output_dir)
    
    tokenizer.save_pretrained(training_args.output_dir)
    trainer.save_model(training_args.output_dir)


if __name__ == "__main__":
    train()


Writing src/Baichuan2/fine-tune/fine-tune.py


In [6]:
%%writefile ./src/ds-train-dist.sh
#!/bin/bash
export WANDB_API_KEY="298b59ce8a416fd45b5fa9ffc17fe72327854e0c"
export WANDB_WATCH="all"
export WANDB_PROJECT="baichuan"

CURRENT_HOST="${SM_CURRENT_HOST}"


IFS=',' read -ra hosts_array <<< "${SM_HOSTS}"
NNODES=${#hosts_array[@]}
NODE_RANK=0

for i in "${!hosts_array[@]}"; do
    if [[ "${hosts_array[$i]}" == *${CURRENT_HOST}* ]]; then
        echo "host index：$i"
        NODE_RANK="$i" 
    fi
done
   
    
MASTER_PORT="13579"
export NCCL_SOCKET_IFNAME="eth0"

#Configure the distributed arguments for torch.distributed.launch.
GPUS_PER_NODE="$SM_NUM_GPUS"
DISTRIBUTED_ARGS="--nproc_per_node $GPUS_PER_NODE \
                  --nnodes $NNODES \
                  --node_rank $NODE_RANK \
                  --master_addr $MASTER_ADDR \
                  --master_port $MASTER_PORT"

chmod +x ./s5cmd
./s5cmd sync s3://$MODEL_S3_BUCKET/llm/models/baichuan2/baichuan-inc/Baichuan2-7B-Base/* /tmp/baichuan2/

    
DEEPSPEED_OPTS="""
    Baichuan2/fine-tune/fine-tune.py 
    --deepspeed Baichuan2/fine-tune/ds_config.json 
    --model_name_or_path "/tmp/baichuan2/" 
    --data_path "Baichuan2/fine-tune/data/belle_chat_ramdon_10k.json" 
    --output_dir "/tmp/baichuan2_out" 
    --num_train_epochs 1 
    --per_device_train_batch_size 1 
    --per_device_eval_batch_size  1 
    --gradient_accumulation_steps 1 
    --evaluation_strategy "no" 
    --save_strategy "steps" 
    --save_steps 2000 
    --save_total_limit 2 
    --learning_rate 2e-5 
    --adam_beta1 0.9 
    --adam_beta2 0.98 
    --adam_epsilon 1e-8 
    --weight_decay 1e-4 
    --max_grad_norm 1.0 
    --warmup_ratio 0.0 
    --lr_scheduler_type "constant" 
    --logging_steps 1 
    --cache_dir '/tmp' 
    --model_max_length 512 
    --gradient_checkpointing True 
    --bf16 True 
    --report_to "wandb"
"""    

CMD="torchrun ${DISTRIBUTED_ARGS} ${DEEPSPEED_OPTS}"
echo ${CMD}
${CMD} 2>&1 

if [[ "${CURRENT_HOST}" == "${MASTER_ADDR}" ]]; then  
    ./s5cmd sync /tmp/baichuan2_out s3://$MODEL_S3_BUCKET/llm/models/baichuan2/output/baichuan-inc/Baichuan-7B-Base/$(date +%Y-%m-%d-%H-%M-%S)/
fi

Overwriting ./src/ds-train-dist.sh


In [7]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-baichuan2-7b-base-finetune:latest'

In [8]:
import time
from sagemaker.estimator import Estimator


environment = {
    'MODEL_S3_BUCKET': sagemaker_default_bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'baichuan2-7b-base-finetune'

instance_type = 'ml.p4d.24xlarge'
# instance_type = 'ml.g5.12xlarge'

estimator = Estimator(role=role,
                      entry_point='ds-train-dist.sh',
                      source_dir='./src',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False)


# estimator.fit(inputs)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:
estimator.fit()

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: baichuan2-7b-base-finetune-2023-10-23-00-36-29-218


2023-10-23 00:36:32 Starting - Starting the training job...
2023-10-23 00:36:47 Pending - Training job waiting for capacity...
2023-10-23 00:37:22 Pending - Preparing the instances for training........................
2023-10-23 00:41:28 Downloading - Downloading input data
2023-10-23 00:41:28 Training - Downloading the training image..............................
2023-10-23 00:46:25 Training - Training image download completed. Training in progress............bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-10-23 00:48:12,310 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-10-23 00:48:12,367 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-10-23 00:48:12,374 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-10-23 00:48:12,376 sagemaker_pytorch_container.training INFO     Invoking use


### Serve large models on SageMaker with DJL DeepSpeed Container

In this notebook, we explore how to host a large language model on SageMaker using the latest container launched using from DeepSpeed and DJL. DJL provides for the serving framework while DeepSpeed is the key sharding library we leverage to enable hosting of large models.We use DJLServing as the model serving solution in this example. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent blog post (https://aws.amazon.com/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/).

Model parallelism can help deploy large models that would normally be too large for a single GPU. With model parallelism, we partition and distribute a model across multiple GPUs. Each GPU holds a different part of the model, resolving the memory capacity issue for the largest deep learning models with billions of parameters. This notebook uses tensor parallelism techniques which allow GPUs to work simultaneously on the same layer of a model and achieve low latency inference relative to a pipeline parallel solution.

SageMaker has rolled out DeepSpeed container which now provides users with the ability to leverage the managed serving capabilities and help to provide the un-differentiated heavy lifting.

In this notebook, we deploy the open source llama 7B model across GPU's on a ml.g5.48xlarge instance. Note that the llama 7B fp16 model can be deployed on single GPU such as g5.2xlarge (24GB VRAM), we jsut show the code which can deploy the llm accross multiple GPUs in SageMaker. DeepSpeed is used for tensor parallelism inference while DJLServing handles inference requests and the distributed workers. For further reading on DeepSpeed you can refer to https://arxiv.org/pdf/2207.00032.pdf 


## Create SageMaker compatible Model artifact and Upload Model to S3

SageMaker needs the model to be in a Tarball format. In this notebook we are going to create the model with the Inference code to shorten the end point creation time. 

The tarball is in the following format

```
code
├──── 
│   └── model.py
│   └── requirements.txt
│   └── serving.properties

```


- `model.py` is the key file which will handle any requests for serving. 
- `requirements.txt` has the required libraries needed to be installed when the container starts up.
- `serving.properties` is the script that will have environment variables which can be used to customize model.py at run time.


#### Serving.properties has engine parameter which tells the DJL model server to use the DeepSpeed engine to load the model.

option.tensor_parallel_degree:  if we use the g5.48xlarge which has 8 GPUs, so we set the tensor_parallel_degree to 8.

option.s3url:  you should use your model path here. And the s3 path must be ended with "/".

batch_size:   it is for server side batch based on request level. You can set batch_size to the large value which can not result in the OOM. The current code about model.py is just demo for one prompt per client request.

max_batch_delay:   it is counted by millisecond. 

In [ ]:
!rm -rf src
!mkdir src

**Copy python files from original model to finetuned model path**

for examples:

!aws s3 cp s3://sagemaker-us-west-2-928808346782/llm/models/baichuan2/baichuan-inc/Baichuan2-7B-Base/ s3://sagemaker-us-west-2-928808346782/llm/models/baichuan/output/baichuan-inc/Baichuan-7B/2023-08-31-06-50-39/baichuan_out/ --recursive --exclude "*" --include "*.py"

In [ ]:
# Copy original *.py files for finetuned model
!aws s3 cp ${s3_url_original_model} ${s3_url_finetuned_model} --recursive --exclude "*" --include "*.py"

**Copy finetune cell output above, modify 'option.s3url' value**

for example, option.s3url=s3://sagemaker-us-west-2-928808346782/llm/models/llama2/output/TheBloke/Llama-2-13B-fp16/2023-08-10-05-32-33/llama_out/

In [ ]:
%%writefile ./src/serving.properties
engine=DeepSpeed
option.tensor_parallel_degree=1
option.s3url=${option.s3url}

In [ ]:
%%writefile ./src/requirements.txt
transformers==4.29.2
sagemaker
nvgpu
xformers
peft

In [ ]:
%%writefile ./src/model.py
from djl_python import Input, Output
import os
import logging
import torch
import deepspeed
import transformers
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer


model = None
tokenizer = None
generator = None


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, torch_dtype=torch.float16, use_fast=False, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)

    return model, tokenizer

 
def handle(inputs: Input) -> None:
    global model, tokenizer
    
    try:
        if not model:
            model, tokenizer = load_model(inputs.get_properties())

        if inputs.is_empty():
            return None

        data = inputs.get_as_json()    

        input_data = data["inputs"]
        params = data["parameters"]

        logging.info(f"data               : {data}")
        logging.info(f"input_data         : {input_data}")
        logging.info(f"params             : {params}")
        
        inputs_tokenized = tokenizer(input_data, return_tensors='pt')
        inputs_tokenized = inputs_tokenized.to('cuda:0')
        logging.info(f"inputs_tokenized   : {inputs_tokenized}")

        pred = model.generate(**inputs_tokenized, **params)
        logging.info(f"pred               : {pred}")
        
        response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=True)
        # response = tokenizer.decode(pred[0], skip_special_tokens=True)
        logging.info(f"response           : {response}")

        result = {"outputs": response}
        logging.info(f"result             : {result}")

        return Output().add({"code":0,"msg":"ok","data":result})
    except Exception as e:
        return Output().add_as_json({"code":-1,"msg":e})

#### Create required variables and initialize them to create the endpoint, we leverage boto3 for this

In [ ]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path


sage_session = sagemaker.Session()
model_bucket = sage_session.default_bucket()  # bucket to house artifacts
s3_code_prefix = (
    "llm/models/baichuan2/code-7b-base"  # folder within bucket where code artifact will go
)

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

**Image URI for the DJL container is being used here**

In [ ]:
#Note that: you can modify the image url according to your specific region.
# inference_image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117"

inference_image_uri = "763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
print(f"Image going to be used is ---- > {inference_image_uri}")

**Create the Tarball and then upload to S3 location**

In [ ]:
!rm -f model.tar.gz
!tar czvf model.tar.gz src

In [ ]:
s3_code_artifact = sage_session.upload_data("model.tar.gz", model_bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

In [ ]:
print(f"S3 Model Bucket is -- > {model_bucket}")

### To create the end point the steps are:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.g5.2xlarge 
    
    b) ContainerStartupHealthCheckTimeoutInSeconds is 15*60 to ensure health check starts after the model is ready
    
3. Create the end point using the endpoint config created    
    

One of the key parameters here is **TENSOR_PARALLEL_DEGREE** which essentially tells the DeepSpeed library to partition the models along 8 GPU's. This is a tunable and configurable parameter.

This parameter also controls the no of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests. For further reading on DeepSpeed you can follow the link https://www.deepspeed.ai/tutorials/inference-tutorial/#initializing-for-inference. 

In [ ]:
from sagemaker.utils import name_from_base


model_name = name_from_base(f"baichuan2-7b-base-finetuned")
print(model_name)

role = sagemaker.get_execution_role()

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact,
    },
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            #"VolumeSizeInGB" : 300,
            #"ModelDataDownloadTimeoutInSeconds": 15*60,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### Wait for the end point to be created.

### This step can take ~ 15 min or longer so please be patient

In [ ]:
import time


resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

#### Leverage the Boto3 to invoke the endpoint. 

This is a generative model so we pass in a Text as a prompt and Model will complete the sentence and return the results


In [ ]:
%%time
import json
import boto3


sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "early_stopping": True,
  "max_new_tokens": 128,
  # "min_new_tokens": 128,
  "do_sample": True,
  # "temperature": 1.0,
}

prompt = "世界上第二高的山峰是哪座"

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs"    : prompt,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

### Delete the endpoint

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)